In [ ]:
from src.raster import Raster
import pandas as pd
import geopandas as gpd
import os

from os import listdir
from os.path import isfile, join

from src.streetview import save_streetview_image, KEY
import pandas as pd
import numpy as np
import re
import time
from django.utils import text
from os import listdir
from os.path import isfile, join

# Get rasters to be labelled

In [ ]:
data_dir = '../data/SplitRaster/'

In [ ]:
images = []
for file in os.listdir(data_dir):
    if file.endswith(".TIF"):
        images.append(file)

parcels = gpd.read_file('../data/Parcels_shape/')

parcel_nums = [int(x.replace('parcel','').replace('.TIF','')) for x in images]
rasters_3000 = parcels.iloc[parcel_nums]

In [ ]:
rasters_3000 = rasters_3000[['MBL', 'AddNum', 'Street']]
rasters_3000.head()

# Get streetview to be labelled

In [ ]:
address_list = pd.read_csv('../data/address_MBL.csv', index_col=0)
address_list.head()

# Create New Labelling List

In [ ]:
to_label_3000 = address_list.merge(rasters_3000, on = 'MBL', how = 'inner')

already_labelled = pd.read_csv("../data/training/all_labels.csv", index_col = 0)
MBLs = list(already_labelled.MBL)
to_label_3000 = to_label_3000[~to_label_3000.MBL.isin(MBLs)]

to_label_3000.to_csv('../data/training3000/to_label_3000.csv')

# Delete Extra Street View

In [ ]:
def clean_address(adr, slugify = True):
    street_adr = adr.split(',')[0]
    fn = re.sub(' +','_',street_adr)
    if slugify:
        fn = text.slugify(fn)
    return fn + '.jpg'

In [ ]:
sv_to_label = to_label_3000.SITE_ADDR.apply(clean_address)
all_sv = [f for f in listdir("../data/training3000/sv_images/") 
                           if isfile(join("../data/training3000/sv_images/", f))]


In [ ]:
to_delete_sv = set(all_sv) - set(sv_to_label)

In [ ]:
for file in to_delete_sv:
    os.remove("../data/training3000/sv_images/" + file)

# Delete Extra Raster

In [ ]:
def get_name(row):
    if row['AddNum'] is None:
        row['AddNum'] = 'unknown'
    if row['Street'] is None:
        row['Street'] = 'unknown'
    address = str(row['AddNum']) + ' ' +row['Street']
    address_cln = address.lower().replace('  ',' ').replace(' ','_')
    filename = address_cln + '_aerial.png'
    return(filename)

In [ ]:
raster_to_label = to_label_3000.apply(get_name, axis = 1)

In [ ]:
all_raster = [f for f in listdir("../data/training3000/aerial_images/") 
                           if isfile(join("../data/training3000/aerial_images/", f))]

In [ ]:
to_delete_raster = set(all_raster) - set(raster_to_label)

In [ ]:
for file in to_delete_raster:
    os.remove("../data/training3000/aerial_images/" + file)